In [100]:
import pandas as pd
import time
import numpy as np
import nba_api as nbp
from nba_api.stats.endpoints import leaguedashplayerstats, playerdashptshots
import requests, os

In [2]:
league_stats = leaguedashplayerstats.LeagueDashPlayerStats().get_data_frames()[0]

In [3]:
#72 game season
league_stats = league_stats.loc[league_stats['GP'] > 50.4] #70% of games
league_stats = league_stats.loc[league_stats['FG3M'] >= 72] #70% of games

In [4]:
pids = league_stats['PLAYER_ID'].to_numpy()
tids = league_stats['TEAM_ID'].to_numpy()
names = league_stats['PLAYER_NAME'].to_numpy()
player_list = [list(i) for i in zip(pids, tids, names)]

In [ ]:
shooting_dict = {}
for p_id, t_id, name in player_list:
    shooting_df = playerdashptshots.PlayerDashPtShots(t_id, p_id).get_data_frames()[4]
    c_make = shooting_df['FG3M'][0] + shooting_df['FG3M'][1]
    c_att = shooting_df['FG3A'][0] + shooting_df['FG3A'][1]
    o_make = shooting_df['FG3M'][2] + shooting_df['FG3M'][3]
    o_att = shooting_df['FG3A'][2] + shooting_df['FG3A'][3]
    shooting_dict[name] = [c_make, c_att, c_make/c_att, o_make, o_att, o_make/o_att]
    time.sleep(.600)

<ipython-input-5-807c0aacf5b6>:8: RuntimeWarning: invalid value encountered in longlong_scalars
  shooting_dict[name] = [c_make, c_att, c_make/c_att, o_make, o_att, o_make/o_att]


In [10]:
full_shooting_df = pd.DataFrame.from_dict(shooting_dict).transpose()
full_shooting_df.columns = ["C FG3M", 'C FG3A', 'C FG3%', "O FG3M", 'O FG3A', 'O FG3%']

In [92]:
full_shooting_df.to_csv("contested_shooting_data.csv")

In [88]:
np.mean(full_shooting_df['C FG3%']) #0.294513523624833

0.294513523624833

In [90]:
np.mean(full_shooting_df['O FG3%']) #0.39410248685329824

0.39410248685329824

In [81]:
games = []
games.append([league_stats.loc[league_stats['PLAYER_NAME'] == name]['GP'].to_list()[0] for name in full_shooting_df.index.values])
games = games[0]

In [94]:
filtered_df = full_shooting_df[full_shooting_df['C FG3A'] >= games] #at least 1 attempted contested 3 per game
filtered_df.to_csv("filtered_contested_shooting_data.csv")

In [120]:
def getHeadshotById(lastName, firstName, folder=None, fileName=None):
    
    if not fileName:
        fileName = str(firstName) + str(lastName) + ".png"

    if not folder:
        folder = ""
    
    url = "https://nba-players.herokuapp.com/players/" + str(lastName) + "/" + str(firstName)
    
    if not os.path.isdir(folder) and folder is not "":
        os.mkdir(folder)
        
    r = requests.get(url, allow_redirects=True)
    if r.ok:
        open(folder + fileName, 'wb').write(r.content)
    else:
        print("Error: Not a valid id. Please specify a valid id.")

<>:11: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:11: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-120-8ac271eaa9ed>:11: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if not os.path.isdir(folder) and folder is not "":


In [121]:
for name in filtered_df.index.values:
    first = str.split(name)[0]
    last = str.split(name)[1]
    getHeadshotById(last, first, folder="headshots/")
    #time.sleep(.600)